In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 4000
import datetime as dt
import re

In [77]:
flags1 = pd.read_csv('akram_files/timing_test_events1.txt', delimiter = '\t',\
                   header = None, names = ['pin_num', 'label', 'date', 'time'])
flags2 = pd.read_csv('akram_files/timing_test_events2.txt', delimiter = '\t',\
                   header = None, names = ['pin_num', 'label', 'date', 'time'])

In [78]:
flags = pd.concat([flags1, flags2])
flags.tail()

,pin_num,label,date,time
410,sync summary event # 6,describe_trial_choice_start,03/06/2023,04:06:46.555 PM
411,sync summary event # 128,async summary event 128,03/06/2023,04:06:46.586 PM
412,sync summary event # 10,describe_trial_choice_end,03/06/2023,04:06:47.732 PM
413,sync summary event # 14,fixation,03/06/2023,04:06:48.767 PM
414,sync summary event # 17,end_experiment,03/06/2023,04:06:49.810 PM


In [79]:
flags['time'] = pd.to_datetime(flags['time'])
flags.head()

,pin_num,label,date,time
0,BioLab,Acquisition Started,03/06/2023,2023-03-16 15:35:06.294
1,sync summary event # 13,start_experiment,03/06/2023,2023-03-16 15:35:17.438
2,sync summary event # 128,async summary event 128,03/06/2023,2023-03-16 15:35:17.469
3,sync summary event # 2,describe_trial_start,03/06/2023,2023-03-16 15:35:26.346
4,sync summary event # 128,async summary event 128,03/06/2023,2023-03-16 15:35:26.417


In [80]:
microsec = []
for row in flags.time:
    microsec.append(row.microsecond)
flags['microsec'] = microsec
flags.head()

,pin_num,label,date,time,microsec
0,BioLab,Acquisition Started,03/06/2023,2023-03-16 15:35:06.294,294000
1,sync summary event # 13,start_experiment,03/06/2023,2023-03-16 15:35:17.438,438000
2,sync summary event # 128,async summary event 128,03/06/2023,2023-03-16 15:35:17.469,469000
3,sync summary event # 2,describe_trial_start,03/06/2023,2023-03-16 15:35:26.346,346000
4,sync summary event # 128,async summary event 128,03/06/2023,2023-03-16 15:35:26.417,417000


In [81]:
flags['label'] = flags['label'].replace('async summary event 128', 'photo_trigger')
flags.head()

,pin_num,label,date,time,microsec
0,BioLab,Acquisition Started,03/06/2023,2023-03-16 15:35:06.294,294000
1,sync summary event # 13,start_experiment,03/06/2023,2023-03-16 15:35:17.438,438000
2,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:17.469,469000
3,sync summary event # 2,describe_trial_start,03/06/2023,2023-03-16 15:35:26.346,346000
4,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:26.417,417000


In [82]:
def clean_the_triggers(df):
    ''' 
    Keep only rows with triggers that are followed by a photo trigger.

    Arguments:
        df: dataframe with triggers

    Returns: dataframe with triggers that have corresponding
        photo triggers only.
    '''

    clean_triggers = pd.DataFrame({'pin_num': [], 'label': [], 'date': [], \
                                   'time': [], 'microsec': []})
    for loc in range(0, df.shape[0] - 1):
        if df.iloc[loc + 1].label == 'photo_trigger':
            clean_triggers = pd.concat([clean_triggers, df.iloc[[loc, loc+1],]])
    return clean_triggers

In [83]:
clean_triggers = clean_the_triggers(flags)
clean_triggers.head()

,pin_num,label,date,time,microsec
1,sync summary event # 13,start_experiment,03/06/2023,2023-03-16 15:35:17.438,438000.0
2,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:17.469,469000.0
3,sync summary event # 2,describe_trial_start,03/06/2023,2023-03-16 15:35:26.346,346000.0
4,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:26.417,417000.0
5,sync summary event # 6,describe_trial_choice_start,03/06/2023,2023-03-16 15:35:36.385,385000.0


In [84]:
clean_triggers['label'].unique()

array(['start_experiment', 'photo_trigger', 'describe_trial_start',
       'describe_trial_choice_start', 'imagine_trial_start',
       'imagine_trial_choice_start', 'describe_trial_choice_end',
       'describe_writeout_start', 'describe_writeout_choice_start',
       'imagine_writeout_start', 'imagine_writeout_choice_start',
       'imagine_writeout_choice_end', 'describe_writeout_choice_end'],
      dtype=object)

In [85]:
def subset_triggers(df, label_of_interest):
    '''
    Pull out rows from original df with
      triggers of interest and their corresponding photo triggers.
    
    Arguments:
        df: dataframe with all the triggers
        label_of_interest: trigger label to pull out.

    Returns:
        Dataframe with the data for the trigger of interest and 
        their corresponding photo triggers.
    '''
    subset_df = pd.DataFrame({'pin_num': [], 'label': [], 'date': [], \
                              'time': [], 'microsec': []})
    for loc in range(0, df.shape[0] - 1):
        if df.iloc[loc].label == label_of_interest:
            subset_df = pd.concat([subset_df, df.iloc[[loc, loc+1],]])

    return subset_df

In [86]:
def find_stats(df):
    ''' 
    Find the standard deviation of the differences between the triggers and the 
    corresponding photo triggers.

    Arguments:
        df: dataframe with only one trigger type
    
    Returns: standard deviation (float) of the differences.
    '''

    # compute the differences
    diffs = []
    for loc in range(0, df.shape[0] - 1):
        if df.iloc[loc].label != "photo_trigger" and \
            df.iloc[loc + 1].label == "photo_trigger":
            diff = df.iloc[loc + 1].microsec- df.iloc[loc].microsec
            diffs.append(diff)
    
    # compute the mean and sd
    M = np.mean(diffs)
    SD = np.std(diffs)

    return (M, SD)

In [87]:
labels = ['describe_trial_start',
       'describe_trial_choice_start', 'imagine_trial_start',
       'imagine_trial_choice_start',
       'describe_writeout_start', 'describe_writeout_choice_start',
       'imagine_writeout_start', 'imagine_writeout_choice_start']
means = {}
SDs = {}
for label in labels:
    temp_df = subset_triggers(clean_triggers, label)
    M, SD = find_stats(temp_df)
    means[label] = M
    SDs[label] = SD

In [88]:
means

{'describe_trial_start': 46267.857142857145,
 'describe_trial_choice_start': 20200.0,
 'imagine_trial_start': -24508.77192982456,
 'imagine_trial_choice_start': 2368.4210526315787,
 'describe_writeout_start': 65333.333333333336,
 'describe_writeout_choice_start': -127333.33333333333,
 'imagine_writeout_start': 35545.454545454544,
 'imagine_writeout_choice_start': 37375.0}

In [89]:
SDs

{'describe_trial_start': 187377.149379784,
 'describe_trial_choice_start': 132669.98016267145,
 'imagine_trial_start': 255859.82475381563,
 'imagine_trial_choice_start': 183726.59240609687,
 'describe_writeout_start': 17518.244457961217,
 'describe_writeout_choice_start': 363462.36241031735,
 'imagine_writeout_start': 8574.418998219342,
 'imagine_writeout_choice_start': 9860.749210886564}

In [91]:
test = subset_triggers(clean_triggers, 'imagine_trial_start')
test

,pin_num,label,date,time,microsec
9,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:35:42.031,31000.0
10,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:42.083,83000.0
83,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:00.015,15000.0
84,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:00.060,60000.0
89,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:13.136,136000.0
90,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:13.176,176000.0
107,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:44.914,914000.0
108,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:44.958,958000.0
119,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:39:10.767,767000.0
120,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:39:10.807,807000.0


In [95]:
diffs = []
for loc in range(0, test.shape[0] - 1):
    if test.iloc[loc].label != "photo_trigger" and \
        test.iloc[loc + 1].label == "photo_trigger":
        diff = test.iloc[loc + 1].microsec - test.iloc[loc].microsec
        diffs.append(diff)

diffs

[52000.0,
 45000.0,
 40000.0,
 44000.0,
 40000.0,
 57000.0,
 64000.0,
 57000.0,
 44000.0,
 29000.0,
 39000.0,
 34000.0,
 64000.0,
 69000.0,
 42000.0,
 63000.0,
 46000.0,
 53000.0,
 41000.0,
 -945000.0,
 45000.0,
 30000.0,
 -954000.0,
 50000.0,
 44000.0,
 50000.0,
 40000.0,
 44000.0,
 56000.0,
 48000.0,
 47000.0,
 60000.0,
 49000.0,
 40000.0,
 42000.0,
 -961000.0,
 29000.0,
 41000.0,
 43000.0,
 -961000.0,
 59000.0,
 49000.0,
 61000.0,
 55000.0,
 43000.0,
 45000.0,
 34000.0,
 40000.0,
 33000.0,
 36000.0,
 47000.0,
 41000.0,
 35000.0,
 36000.0,
 44000.0,
 39000.0,
 46000.0]

In [97]:
sec = []
for row in test.time:
    sec.append(row.second)
test['sec'] = sec
test

,pin_num,label,date,time,microsec,sec
9,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:35:42.031,31000.0,42
10,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:42.083,83000.0,42
83,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:00.015,15000.0,0
84,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:00.060,60000.0,0
89,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:13.136,136000.0,13
90,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:13.176,176000.0,13
107,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:44.914,914000.0,44
108,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:44.958,958000.0,44
119,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:39:10.767,767000.0,10
120,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:39:10.807,807000.0,10


In [98]:
test['blah'] = test.sec + test.microsec
test.head()

,pin_num,label,date,time,microsec,sec,blah
9,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:35:42.031,31000.0,42,31042.0
10,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:35:42.083,83000.0,42,83042.0
83,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:00.015,15000.0,0,15000.0
84,sync summary event # 128,photo_trigger,03/06/2023,2023-03-16 15:38:00.060,60000.0,0,60000.0
89,sync summary event # 1,imagine_trial_start,03/06/2023,2023-03-16 15:38:13.136,136000.0,13,136013.0
